In the previous file we had the features for our graph neural network, now time to use D-GCN(Deep Graph Convulation Network) for doing it<br>
The features of this includes
1. A GCN that takes the graph features as inputs
2. A global Attention Encoder to project it into a latent space
3. A full model that combines the GCn and the Global Attention Encoder

# Importing Libraries

In [ ]:
!pip install torch-geometric rdkit-pypi

In [8]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.data import Data
from torch_geometric.nn import GATConv
from torch_geometric.data import DataLoader

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [24]:
from sklearn.manifold import TSNE

# Features File

In [16]:
def featurize_molecule(mol):
    # Compute Morgan fingerprints for each atom
    atom_features = []
    for atom in mol.GetAtoms():
        idx = atom.GetIdx()
        atom_feature = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, atomIndices=[idx])
        atom_features.append(np.array(atom_feature))

    return np.array(atom_features)

In [101]:
def smiles_to_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)

    # Add explicit hydrogens
    mol = Chem.AddHs(mol)

    # Generate 3D coordinates for visualization
    AllChem.EmbedMolecule(mol, randomSeed=42)  # You can choose any seed value

    # Get atom features and adjacency matrix
    num_atoms = mol.GetNumAtoms()
    atom_features = np.zeros((num_atoms, 3))  # You may need to adjust the feature dimensions
    adjacency_matrix = np.zeros((num_atoms, num_atoms))

    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        adjacency_matrix[i, j] = adjacency_matrix[j, i] = 1  # Adjacency matrix is symmetric

    for atom in mol.GetAtoms():
        idx = atom.GetIdx()
        atom_features[idx, 0] = atom.GetAtomicNum()  # Atom type or atomic number
        atom_features[idx, 1] = atom.GetTotalNumHs()  # Number of hydrogen atoms
        atom_features[idx, 2] = atom.GetFormalCharge()  # Formal charge

    # Convert to PyTorch tensors
    atom_features = torch.tensor(atom_features, dtype=torch.float)

    # Create edge_index using the adjacency matrix
    edge_index = torch.tensor(np.column_stack(np.where(adjacency_matrix)), dtype=torch.long)

    # Create PyTorch Geometric data object
    data = Data(x=atom_features, edge_index=edge_index.t().contiguous())  # Transpose edge_index

    return data

# GCN
Graph Convulation Network For processing the input features with the use of attention

In [9]:
class MultiHeadAttention(nn.Module):
    def __init__(self, in_channels, out_channels, attention_heads=1):
        super().__init__()
        self.W_q = nn.Linear(in_channels, out_channels)
        self.W_k = nn.Linear(in_channels, out_channels)
        self.W_v = nn.Linear(in_channels, out_channels)
        self.attention_heads = attention_heads

    def forward(self, x):
        # Apply linear transformations to obtain queries, keys, and values
        q = self.W_q(x)
        k = self.W_k(x)
        v = self.W_v(x)

        # Reshape queries, keys, and values for multi-head attention
        q = q.view(-1, self.attention_heads, q.size(-1))
        k = k.view(-1, self.attention_heads, k.size(-1))
        v = v.view(-1, self.attention_heads, v.size(-1))

        # Compute scaled dot-product attention
        scores = torch.matmul(q, k.transpose(-2, -1)) / torch.sqrt(torch.tensor(q.size(-1), dtype=torch.float32))
        attention_weights = F.softmax(scores, dim=-1)
        attention_output = torch.matmul(attention_weights, v).view(x.size(0), -1)

        return attention_output

In [32]:
class GATModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=1, external_attention_heads=None):
        super(GATModel, self).__init__()
        self.conv1 = GATConv(in_channels, hidden_channels, heads=heads)
        self.external_attention = MultiHeadAttention(hidden_channels * heads, hidden_channels, attention_heads=external_attention_heads)
        self.conv2 = GATConv(hidden_channels * heads, out_channels, heads=1)
        self.external_attention_heads=external_attention_heads

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # First GAT layer
        x = self.conv1(x, edge_index)
        x = torch.relu(x)

        if self.external_attention_heads is not None:

          # External Attention
          external_attention_output = self.external_attention(x)

          # Concatenate GAT output and external attention output
          x = torch.cat([x, external_attention_output], dim=-1)

        else:
          x=x

        # Second GAT layer
        x = self.conv2(x, edge_index)

        return x

# Global Attention Encoder

The Graph Attention Encoder here is making the use of positional embeddings with the help of pairwise distance between the adjacency matrix

In [99]:
class PositionalEncoding(nn.Module):
    def __init__(self, input_size, max_len=1000):
        super(PositionalEncoding, self).__init__()
        self.encoding = nn.Embedding(max_len, input_size)

    def forward(self, x):
        positions = torch.arange(0, x.size(1), device=x.device).unsqueeze(0)
        positions = positions.expand(x.size(0), -1)  # Expand along batch dimension
        return x + self.encoding(positions)

In [100]:
class DistanceAttentionEncoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(DistanceAttentionEncoder, self).__init__()

        self.embedding = PositionalEncoding(input_size)
        self.encoder = nn.Linear(input_size, hidden_size)
        self.decoder = nn.Linear(hidden_size, 1)
        self.softmax = nn.Softmax(dim=1)

    def pairwise_distances(self, x):
        # Calculate pairwise distances using L2 norm
        distances = torch.norm(x[:, None, :] - x, dim=-1, p=2)
        return distances

    def forward(self, input_sequence):
        # Assuming input_sequence has shape (batch_size, sequence_length, input_size)

        # Apply positional embeddings
        embedded_sequence = self.embedding(input_sequence)

        # Encode the embedded sequence
        encoded_sequence = self.encoder(embedded_sequence)

        # Calculate attention scores
        attention_scores = self.decoder(torch.tanh(encoded_sequence))

        # Apply softmax to get attention weights
        attention_weights = self.softmax(attention_scores)

        # Apply attention weights to the encoded sequence
        context_vector = torch.sum(encoded_sequence * attention_weights, dim=1)

        return context_vector

# Making a Full Pipeline
Now making a full pipeline that will contain the GCN And the Global Attention Encoder

In [95]:
class GATModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=1, external_attention_heads=None):
        super(GATModel, self).__init__()

        self.conv1 = GATConv(in_channels, hidden_channels, heads=heads)
        self.external_attention = MultiHeadAttention(hidden_channels * heads, hidden_channels, attention_heads=external_attention_heads)
        self.conv2 = GATConv(hidden_channels * heads, out_channels, heads=1)
        self.external_attention_heads = external_attention_heads



    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # First GAT layer
        x = self.conv1(x, edge_index)
        x = torch.relu(x)

        if self.external_attention_heads is not None:
            # External Attention
            external_attention_output = self.external_attention(x)

            # Concatenate GAT output and external attention output
            x = torch.cat([x, external_attention_output], dim=-1)



        # Second GAT layer
        x = self.conv2(x,edge_index)


        self.distance_attention_encoder = DistanceAttentionEncoder(x.size(1), hidden_size=64)

        # Apply distance attention encoder
        distance_attention_output = self.distance_attention_encoder(x.unsqueeze(0))

        return x

# Example Usage

In [102]:
# Example usage:
smiles_string = "CN1C=NC2=C1C(=O)N(C(=O)N2C)C"
graph_data = smiles_to_graph(smiles_string)
print('Graph Data Shape:', graph_data.x.size())

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the GAT model
in_channels = graph_data.x.size(1)  # Number of input features
hidden_channels = 64
out_channels = 32
heads = 2  # Number of attention heads
gat_model = GATModel(in_channels, hidden_channels, out_channels, heads).to(device)

# Forward pass
output = gat_model(graph_data)

# Print the output shape
print("Output shape:", output.shape)

Graph Data Shape: torch.Size([24, 3])
Output shape: torch.Size([24, 32])
